In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import gc
import h5py
import os
import cv2
import numpy as np
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [5]:
def moodNamePrintFromLabel(n):
  if n == 0: result = 'angry '
  elif n == 1: result = 'disgust '
  elif n == 2: result = 'fear'
  elif n == 3: result = 'happy'
  elif n == 4: result = 'sad'
  elif n == 5: result = 'surprise'
  elif n == 6: result = 'neutral'
  return result

In [6]:
def load_images_from_directory(directory, target_size=(48, 48)):
    images = []
    labels = []
    label_map = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'sad': 4, 'surprise': 5, 'neutral': 6}
    for label_name, label_index in label_map.items():
        class_dir = os.path.join(directory, label_name)
        if not os.path.isdir(class_dir):
            continue
        for img_name in tqdm(os.listdir(class_dir)):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, target_size)
                images.append(img.reshape(target_size[0], target_size[1], 1))
                labels.append(label_index)
    return np.array(images), np.array(labels)


# Load FER2013 images
X_fer2013, y_fer2013 = load_images_from_directory('Test Image Data/FER-2013')

# Load AffectNet images
X_affectnet, y_affectnet = load_images_from_directory('Test Image Data/AffectNet')

# Load RAF-DB images
X_RAF, y_RAF = load_images_from_directory('Test Image Data/RAF-DB')

100%|██████████| 680/680 [00:00<00:00, 6270.53it/s]


In [10]:

def preprocess_image(image, target_size=(224, 224)):
    image = image.astype(np.float32)
    image = np.clip(image, 0, 255).astype(np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = cv2.resize(image, target_size)
    image = (image / 127.5) - 1
    return image

def preprocess_and_save_images_with_labels(images, labels, output_file, dataset_name_image, dataset_name_label, batch_size=50):
    num_batches = len(images) // batch_size + (1 if len(images) % batch_size != 0 else 0)
    with h5py.File(output_file, 'w') as hf:
        # Create datasets for images and labels
        processed_shape = (len(images), 224, 224, 3)
        dataset_image = hf.create_dataset(dataset_name_image, shape=processed_shape, dtype=np.float32)
        dataset_label = hf.create_dataset(dataset_name_label, data=labels, dtype=np.uint8)
        
        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, len(images))
            batch_images = images[start_idx:end_idx]
            processed_batch_images = np.array([preprocess_image(image) for image in batch_images], dtype=np.float32)
            dataset_image[start_idx:end_idx] = processed_batch_images
            del batch_images, processed_batch_images
            gc.collect()




# Preprocess images in batches and save to HDF5 files with uniform names
preprocess_and_save_images_with_labels(X_fer2013, y_fer2013, 'output files/fer2013_testing.h5', 'images', 'labels')
preprocess_and_save_images_with_labels(X_affectnet, y_affectnet, 'output files/affectnet_testing.h5', 'images', 'labels')
preprocess_and_save_images_with_labels(X_RAF, y_RAF, 'output files/rafdb_testing.h5', 'images', 'labels')

# Check the shapes of the processed arrays
with h5py.File('output files/fer2013_testing.h5', 'r') as hf:
    fer2013_images = hf['images']
    print("Shape of FER2013 images:", fer2013_images.shape)

with h5py.File('output files/affectnet_testing.h5', 'r') as hf:
    affectnet_images = hf['images']
    print("Shape of AffectNet images:", affectnet_images.shape)

with h5py.File('output files/rafdb_testing.h5', 'r') as hf:
    rafdb_images = hf['images']
    print("Shape of Rafdb images:", rafdb_images.shape)

Shape of FER2013 images: (7178, 224, 224, 3)
Shape of AffectNet images: (0, 224, 224, 3)
Shape of Rafdb images: (3068, 224, 224, 3)
